In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Caminho para o arquivo JSON da conta de serviço
key_path = r"C:\BIGQUERY_KEY_PATH\rock-tower-454219-t2-f2eeba73d76a.json"

# Carregar as credenciais
credentials = service_account.Credentials.from_service_account_file(key_path)

# Criar o cliente BigQuery com as credenciais
client = bigquery.Client(project='rock-tower-454219-t2', credentials=credentials, location='US')

# Exemplo de consulta
query = """
   SELECT
    dados.ano as ano,
    dados.sigla_uf AS sigla_uf,
    diretorio_sigla_uf.nome AS sigla_uf_nome,
    dados.id_municipio AS id_municipio,
    diretorio_id_municipio.nome AS id_municipio_nome,
    dados.bairro_revenda as bairro_revenda,
    dados.cep_revenda as cep_revenda,
    dados.endereco_revenda as endereco_revenda,
    dados.cnpj_revenda as cnpj_revenda,
    dados.nome_estabelecimento as nome_estabelecimento,
    dados.bandeira_revenda as bandeira_revenda,
    dados.data_coleta as data_coleta,
    dados.produto as produto,
    dados.unidade_medida as unidade_medida,
    dados.preco_compra as preco_compra,
    dados.preco_venda as preco_venda
FROM `basedosdados.br_anp_precos_combustiveis.microdados` AS dados
LEFT JOIN (SELECT DISTINCT sigla,nome  FROM `basedosdados.br_bd_diretorios_brasil.uf`) AS diretorio_sigla_uf
    ON dados.sigla_uf = diretorio_sigla_uf.sigla
LEFT JOIN (SELECT DISTINCT id_municipio,nome  FROM `basedosdados.br_bd_diretorios_brasil.municipio`) AS diretorio_id_municipio
    ON dados.id_municipio = diretorio_id_municipio.id_municipio
"""

# Executar a consulta
job = client.query(query)
results = job.to_dataframe()
print(results)

           ano sigla_uf sigla_uf_nome id_municipio id_municipio_nome  \
0         2022       RO      Rondônia      1100049            Cacoal   
1         2022       RO      Rondônia      1100049            Cacoal   
2         2022       RO      Rondônia      1100122         Ji-Paraná   
3         2022       AC          Acre      1200401        Rio Branco   
4         2022       AC          Acre      1200401        Rio Branco   
...        ...      ...           ...          ...               ...   
15513994  2017       SP     São Paulo      3550308         São Paulo   
15513995  2017       SP     São Paulo      3550308         São Paulo   
15513996  2017       SP     São Paulo      3550308         São Paulo   
15513997  2017       SP     São Paulo      3550308         São Paulo   
15513998  2017       SP     São Paulo      3550308         São Paulo   

             bairro_revenda cep_revenda  \
0                 Teixeirao    76965674   
1                 Teixeirao    76965674   
2     